In [4]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
from cProfile import label

In [3]:
train_df = pd.read_json('../data/relations_dev.txt')
test_df = pd.read_json('../data/relations_test.txt') 

In [5]:
def pre_process_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [6]:
train_df['label'] = train_df['tokens'].apply(pre_process_tokens)
test_df['label'] = test_df['tokens'].apply(pre_process_tokens)

In [7]:
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


####  preprocess the data

In [9]:
train_doc = []
for i in range(train_df.shape[0]):
    ent = train_df.label.iloc[i]
    docu = train_df.document.iloc[i].replace("\n"," ")
    if len(ent)!=0:
        train_doc.append(docu+"\n\nExtracted Text:" +
                        '\n'+ent+"----\n")
    
with open('../data/entity.txt','w') as f:
    for item in train_doc:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Done')

Done


In [10]:
test_doc = test_df.document.apply(
    lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()

In [12]:
import sys
sys.path.insert(0,'../')
import config

In [13]:
import cohere
co = cohere.Client(config.api_key)

#### modeling

In [14]:
class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        prompt = self.examples + [example]
       
        return ("".join([str(exam) for exam in prompt]))

    def extract(self, example):
        extraction = co.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=100,
            temperature=0.5,
            stop_sequences=["----"])
        
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor(train_doc[:6])

In [15]:
print(cohereMovieExtractor.make_prompt(test_doc[-1]))

Bachelor's degree in Mechanical Engineering or Physical Science 3+ years track record of developing or specifying fiber optic cables and connector related products Knowledge of fiber optic component, cabling, and interconnect products, technologies, and standards Experience in statistical data analysis Experience with product life cycle management (PLM) process Experience providing solutions to problems and meeting deadlines Experience engaging stakeholders PREFERRED Advanced degree Experience using a software tool for statistical data analysis such as JMP Experience using Agile as product life-cycle management tool Data center or other mission critical development experience

Extracted Text:
DIPLOMA:Bachelor
DIPLOMA_MAJOR:Mechanical Engineering,Physical Science
EXPERIENCE:3+ years
SKILLS:developing,fiber optic cables,connector related products
----
10+ years of software engineering work experience. Technical experience in release automation engineering, CI/CD or related roles. Experie

In [16]:
results = []
for text in tqdm(test_doc):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

  9%|▉         | 1/11 [00:03<00:39,  3.91s/it]


EXPERIENCE:1+ year
SKILLS:computer science,machine learning,AI,statistics,data science,research
---


 18%|█▊        | 2/11 [00:07<00:33,  3.73s/it]


EXPERIENCE:2+ years
SKILLS:online advertising,research,
---


 27%|██▋       | 3/11 [00:11<00:30,  3.76s/it]

...


EXPERIENCE:12+ years
SKILLS:research experience,human-computer interaction,human factors,cognitive psychology,shipped products
---


 91%|█████████ | 10/11 [00:37<00:03,  3.81s/it]


DIPLOMA:PhD
DIPLOMA_MAJOR:Industrial/Organizational Psychology,Organizational Behavior,Labor Economics,Management,Policy Analysis,
EXPERIENCE:3+ years
SKILLS:quantitative people research,managing,conveying
---


100%|██████████| 11/11 [00:42<00:00,  3.82s/it]


EXPERIENCE:1+ year
SKILLS:AI,research,team
---


In [17]:
pd.DataFrame(data={'text': test_doc, 'extracted_text': results})

,text,extracted_text
0,"Currently holding a faculty, industry, or gov...","\nEXPERIENCE:1+ year\nSKILLS:computer science,..."
1,2+ years experience in the online advertising...,\nEXPERIENCE:2+ years\nSKILLS:online advertisi...
2,BA/BS 5+ years of program or project manageme...,\nEXPERIENCE:5+ years\nSKILLS:program or proje...
3,Currently enrolled in a full-time degree prog...,"\nEXPERIENCE:0 years\nSKILLS:research,machine ..."
4,Currently enrolled in a full-time degree prog...,\nDIPLOMA:Pursuing\nDIPLOMA_MAJOR:Computer Sci...
5,MS in a quantitative field such as Operations...,"\nEXPERIENCE:7+ years\nSKILLS:MS,Quantitative ..."
6,"Ph.D. with 5+ years of experience, MS with 7+...","\nDIPLOMA:Ph.D\nDIPLOMA_MAJOR:Physics,Electric..."
7,Graduating with a Ph.D. in Computer Science o...,\nEXPERIENCE:Graduating with a Ph.D.\nSKILLS:r...
8,"12+ years of research experience, in-house or...",\nEXPERIENCE:12+ years\nSKILLS:research experi...
9,"PhD with 3+ years, or MS with 5+ years of exp...",\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Industrial/Organi...


In [18]:
def clean_test(x):
    x = "".join(x)
    x = x.lower()
    x = x.lstrip()
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

def clean_extracted(x):
    x = "".join(x)
    x = x.lower()
    x = x[:-3]
    x = x.lstrip()
    
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
   
    return x

In [19]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())

In [20]:
# Save results
test_df['extracted_text'] = extracted

In [21]:
test_df['extracted_text'] = test_df['extracted_text'].apply(clean_extracted)
test_df['label'] = test_df['label'].apply(clean_test)

In [22]:
from difflib import SequenceMatcher
score = []
for i in range(test_df.shape[0]):
    
    score.append(SequenceMatcher(None,test_df['label'].iloc[i],
                          test_df['extracted_text'].iloc[i]).ratio())

In [23]:
# Compare the label to the extracted text
test_df['similarity_score'] = pd.DataFrame(score)

# Print the accuracy
print(f'Classification accuracy {test_df["similarity_score"].mean() *100}%')

Classification accuracy 42.207696687784896%


In [24]:
test_df

,document,tokens,relations,label,extracted_text,similarity_score
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","diplomaph.d.diploma_majormachine learning,ai,c...","diplomaph.d.diploma_majormachine learning,ai,c...",0.571429
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...","experience2+ yearsskillsonline advertising,res...","experience2+ yearsskillsresearch,online advert...",0.743494
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","experience5+ years,2+ yearsskillsproject manag...","experience5+ years,2+ yearsskillsprogram,proje...",0.867052
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diplomaph.d.diploma_majorcomputer science,diplomapursuingdiploma_majorcomputer scienceex...,0.385417
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diplomaph.d.diploma_majorcomputer science,diplomapursuing a ph.d. degree in computer sci...,0.251534
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","diplomams,phddiploma_majoroperations research,...","experience7+ years,5+ yearsskillsms,quantitati...",0.056801
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,"experience5+ yearsskillsphysics,electrical eng...",0.000000
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",diplomaph.d.diploma_majorcomputer science,diplomaph.d.diploma_majorcomputer scienceexper...,0.302583
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","experience12+ yearsskillsresearch,ph.ddiploma_...","experience12+ yearsskillsresearch,human-comput...",0.859729
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","experience5+ years,3+ years,3+ years,5+ yearss...",diplomaphddiploma_majorindustrial/organization...,0.330523
